In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels = 4)

In [4]:
from transformers import Trainer, TrainingArguments
import numpy as np
from datasets import load_dataset, load_metric